In [10]:
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from urllib.request import urlopen
import urllib

#from tqdm import tqdm

from selenium import webdriver
import openpyxl
import time
import math
from collections import Counter

import re
import json

from tqdm.notebook import tqdm

In [11]:
def url_split(url):
    main_,para_1 = url.split("?")
    para_2 = para_1.split("&")
    para_3 = [d.replace("=",":").split(":") for d in para_2]
    params_ = {}
    for key,value in para_3:
        params_[key] = value
    return main_,params_

In [12]:
ua = UserAgent()

# Main page 

In [13]:
def Main_page_SSG(Key_,Page_):
    ua = UserAgent()
    header = {'User-Agent':ua.chrome,
              "referer":"http://www.ssg.com/"}
    page_ = Page_
    key_ = Key_

    url = "http://www.ssg.com/search.ssg?target=all&query={}&sort=cnt&page={}".format(key_,page_)
    html = requests.get(url.format(key_),headers=header).text
    soup = BeautifulSoup(html,"html.parser")
    url_base_ = "http://www.ssg.com"
    name_head,url_head,reveiw_count_1 = [],[],[]
    for tag in soup.select("li"):
        try:
            name_head.append(tag.find("div",class_="cunit_info").find("div",class_="cunit_md notranslate").find("a").find("em",class_="tx_ko").text)
            url_head.append(tag.find("div",class_="cunit_info").find("div",class_="cunit_md notranslate").find("a")["href"])
            reveiw_count_1.append(int(tag.find("div",class_="cunit_app").find("span",class_="rate_tx").find("em").text.replace(",","")))
    
        except:
            pass
    Head_df = pd.DataFrame([name_head,reveiw_count_1,url_head]).T.rename(columns={0:"Name",1:"Review_count",2:"Url"})
    return Head_df

In [14]:
def Main_page_concat(Total_p,Key_):
    tq = tqdm(range(Total_p))
    head_all = pd.DataFrame()
    for tp in tq:
        pa_ = tp + 1
        candi_head =  Main_page_SSG(Key_,pa_)
        head_all = pd.concat([head_all,candi_head]).reset_index(drop=True)
    return head_all

In [15]:
head_df = Main_page_concat(1,"헤어드라이기")
head_df

  0%|          | 0/1 [00:00<?, ?it/s]

,Name,Review_count,Url
0,"플라 이지랩 자동 봉고데기 및 드라이기, 고데기, 두피케어~HIT 헤어스타일러 제안",3402,/item/dealItemView.ssg?itemId=1000065874177&si...
1,★[브랜드위크 20% 쿠폰] JMW 항공모터 드라이기/고데기 특가,2709,/item/dealItemView.ssg?itemId=1000039917682&si...
2,★[브랜드위크 20% 쿠폰] JMW 프리미엄 항공모터 드라이기/고데기 특가,2709,/item/dealItemView.ssg?itemId=1000036476134&si...
3,다이슨코리아 슈퍼소닉/에어랩 갓스타일의 완성,1618,/item/dealItemView.ssg?itemId=1000040673169&si...
4,★15%쿠폰할인BLDC항공모터드라이기&고데기모음전,1374,/item/dealItemView.ssg?itemId=1000039492458&si...
...,...,...,...
75,"아름다움의 시작 케어에서부터, JMW 외 이미용가전 모음전",37,/item/dealItemView.ssg?itemId=1000066879600&si...
76,1500W UN-A1141 전문가용드라이기/미용실헤어드라이기/업소용드라이기/음이온/...,None,/item/itemView.ssg?itemId=1000010220834&siteNo...
77,4YOU 다이슨 에어랩 실리콘 커버 케이스 스크래치방지,None,/item/itemView.ssg?itemId=1000051519330&siteNo...
78,프리미엄케어 스튜디오 드라이 HV5461,None,/item/itemView.ssg?itemId=1000060383446&siteNo...


# Review_page

In [16]:
def Review_URL(head_df,target_row):
    """
    head_df = main page information
    target_row = head_df's target product
    page_2 = target product's review pages
    """
    url_base_ = 'http://www.ssg.com'
    Target_name = head_df["Name"][target_row]
    target_url = head_df["Url"][target_row]#.encode('utf-8')
    Target_max_p = head_df["Review_count"][target_row]

    ## Request HTML
    header = {'User-Agent':ua.chrome,
              "referer":"http://www.ssg.com/"}

    html = requests.get(url_base_ + target_url,headers=header).text
    soup = BeautifulSoup(html,"html.parser")

    ## Review_page Params data

    actu_num = 0
    for tag in soup.select(".chd_lst_choice "):
        try:
            actu_num = tag.find("li",class_="item")["data-deal-option"]
        except:
            pass

    dic_ = {}
    for m in [i.split("=") for i in target_url.split("?")[1].split("&")]:
        dic_[m[0]] = m[1]
    dic_["itemId"] = actu_num

    ## Review_page Request

    url_re_base = "http://www.ssg.com/item/ajaxItemCommentList.ssg?itemId={}&siteNo={}&page={}&pageSize={}"
    item_id = dic_["itemId"]
    site_no = dic_["siteNo"]
    page_sz = 20
    Url_review = url_re_base.format(item_id,site_no,"{}",page_sz)
    return Url_review,Target_name,Target_max_p

In [18]:
url_review,target_name,max_page = Review_URL(head_df,1)
url_review

'http://www.ssg.com/item/ajaxItemCommentList.ssg?itemId=1000031479690&siteNo=6004&page={}&pageSize=20'

In [19]:
target_name

'★[브랜드위크 20% 쿠폰] JMW 항공모터 드라이기/고데기 특가'

In [36]:
def Review_page_ex(url_review,target_name):
    header = {'User-Agent':ua.chrome,
              "referer":"http://www.ssg.com/"}

    html = requests.get(url_review,headers=header).text
    review_n = len(json.loads(html)["pageDto"]["resultList"])
    review_1 = json.loads(html)["pageDto"]["resultList"]
    data_all = []
    for re_i in range(len(json.loads(html)["pageDto"]["resultList"])):
        date_ = json.loads(html)["pageDto"]["resultList"][re_i]["wrtDt"]
        id_ = json.loads(html)["pageDto"]["resultList"][re_i]["mbrLoginId"]
        item_ = target_name
        score_ = json.loads(html)["pageDto"]["resultList"][re_i]["recomEvalScr"]
        contents_ = json.loads(html)["pageDto"]["resultList"][re_i]["postngTitleNm"]
        liked_ = ""
        channel_ = "SSG"
        all_ = [date_,id_,item_,score_,contents_,liked_,channel_]
        data_all.append(all_)
    df_ = pd.DataFrame(data_all).rename(columns={0:"date",1:"id",2:"item",3:"score",4:"contents",5:"liked",6:"channel"})
    return df_

In [37]:
def Crawling_SSG_all(head_df,TA_):
    Url_review,target_name,max_page = Review_URL(head_df,TA_)
    MaxPage = max_page//10
    tq = tqdm(range(MaxPage))

    review_all = pd.DataFrame()

    for mp in tq:
        url_review =Url_review.format(mp+1)
    #    print(url_review)
        review_candi = Review_page_ex(url_review,target_name)
        review_all = pd.concat([review_all,review_candi]).reset_index(drop=True)
    return review_all,target_name

In [38]:
out_,NA_ = Crawling_SSG_all(head_df,1)
out_

  0%|          | 0/270 [00:00<?, ?it/s]

,date,id,item,score,contents,liked,channel
0,20210527,cjj*******,★[브랜드위크 20% 쿠폰] JMW 항공모터 드라이기/고데기 특가,5,아기가 드라이기를 자꾸 망가트려서 튼튼하면서도 전자파걱정 없는 제품찾다가 구매했어요...,,SSG
1,20210410,les*******,★[브랜드위크 20% 쿠폰] JMW 항공모터 드라이기/고데기 특가,5,현재 사용중인 헤어드라이기가 10년이 넘게 사용하고 있었지요. 딱히 새로 살 필요성...,,SSG
2,20210712,cha*******,★[브랜드위크 20% 쿠폰] JMW 항공모터 드라이기/고데기 특가,5,바람세기가 low high turbo 3단계로되어있어서\n다양하게 활용가능해요.\n...,,SSG
3,20210707,yel*******,★[브랜드위크 20% 쿠폰] JMW 항공모터 드라이기/고데기 특가,5,믿고 쓰는 JMW 슥닷컴 사은품과 만원 할인 쿠폰까지\n만족한 가격에 구입 했네요...,,SSG
4,20210126,ghk*******,★[브랜드위크 20% 쿠폰] JMW 항공모터 드라이기/고데기 특가,4,가볍고 바람이 정말 세고 좋아요! \r\n단점이라면 바람이 너무 세서 그런지 좀 엉...,,SSG
...,...,...,...,...,...,...,...
1151,20190417,pas*******,★[브랜드위크 20% 쿠폰] JMW 항공모터 드라이기/고데기 특가,5,성능과 디자인 모두 만족스러운 제품,,SSG
1152,20190410,dus*******,★[브랜드위크 20% 쿠폰] JMW 항공모터 드라이기/고데기 특가,5,가성비최고네요무엇보다가벼워서좋아요,,SSG
1153,20190330,lov*******,★[브랜드위크 20% 쿠폰] JMW 항공모터 드라이기/고데기 특가,5,가볍고 바람이 강하면서도 부드러워요~,,SSG
1154,20190215,tty*******,★[브랜드위크 20% 쿠폰] JMW 항공모터 드라이기/고데기 특가,5,포장 잘되어서 오네요\r\n이마트 좋습니다.,,SSG


In [39]:
def SSG_Save_local(path_,head_df):
    for i_ in range(len(head_df)):
        print("In {} Pro From {}".format(i_+1,len(head_df)+1))
        print()
        print(head_df["Name"][i_])
        out_,NA_ = Crawling_SSG_all(head_df,i_)
        NA_ = NA_.replace("/","")
        out_.to_excel(path_.format(NA_))

In [40]:
path_ = r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\hair\shopping_2\SSG\{}.xlsx"

SSG_Save_local(path_,head_df)

In 1 Pro From 81

플라 이지랩 자동 봉고데기 및 드라이기, 고데기, 두피케어~HIT 헤어스타일러 제안


  0%|          | 0/340 [00:00<?, ?it/s]

In 2 Pro From 81

★[브랜드위크 20% 쿠폰] JMW 항공모터 드라이기/고데기 특가


  0%|          | 0/270 [00:00<?, ?it/s]

In 3 Pro From 81

★[브랜드위크 20% 쿠폰] JMW 프리미엄 항공모터 드라이기/고데기 특가


...

  0%|          | 0/3 [00:00<?, ?it/s]

In 76 Pro From 81

아름다움의 시작 케어에서부터, JMW 외 이미용가전 모음전


  0%|          | 0/3 [00:00<?, ?it/s]

In 77 Pro From 81

1500W UN-A1141 전문가용드라이기/미용실헤어드라이기/업소용드라이기/음이온/강약냉풍조절/4단조절


TypeError: unsupported operand type(s) for //: 'NoneType' and 'int'